In [1]:
#pip install beautifulsoup4
#pip install requests

In [2]:
from bs4 import BeautifulSoup
import os
import re
import requests
import time
import zipfile

# Constantes

In [7]:
TRANSPARENCIA_URL = 'http://www.portaltransparencia.gov.br/'
RECEITA_URL = 'download-de-dados/receitas'

In [8]:
# Pega o HTML do site de Receitas
response = requests.get(TRANSPARENCIA_URL+RECEITA_URL)
print(response.status_code)

200


In [9]:
# Resultado
print(response.content)

b'\n\n\n\n<!DOCTYPE HTML>\n<html lang="pt-br">\n<head>\n    <meta http-equiv="Content-Type"     content="text/html; charset=UTF-8">\n    <meta http-equiv="X-UA-Compatible"  content="IE=edge">\n    <meta name="viewport"               content="width=device-width,initial-scale=1.0">\n    <meta name="description"            content="">\n    <meta name="author"                 content=""/>\n    <title>Execu\xc3\xa7\xc3\xa3o da Receita - Portal da transpar\xc3\xaancia</title>\n\n    <!-- Global site tag (gtag.js) - Google Analytics -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-1665737-25"></script>\n    <script>\n        window.dataLayer = window.dataLayer || [];\n        function gtag(){dataLayer.push(arguments);}\n        gtag(\'js\', new Date());\n\n        gtag(\'config\', \'UA-1665737-25\');\n    </script>\n\n    <!-- BARRA DO TOPO DO GOVERNO -->\n    <link rel="stylesheet"              href="/static/libs/font-awesome-4.4.0/css/font-awesome.min.css?v=1.4.2" 

## Usa o beautiful soup para manusear o resultado do HTML mais facilmente desde que a página tenha sido renderizada

In [10]:
soup = BeautifulSoup(response.content)
soup.title

<title>Execução da Receita - Portal da transparência</title>

In [11]:
soup.p

<p>Selecione o exercício desejado e clique em Baixar.</p>

In [12]:
soup.find(id="links-anos")

<select id="links-anos"></select>

In [13]:
anos=soup.find("select",{"id":"links-anos"})
options = anos.find_all("option")
values = [o.get("value") for o in options]
for x in values:
    print(x)

## Como a página possui javascript, podemos pegar os campos desejado por REGEX

In [14]:
anos = re.findall('\"ano\" : \"([0-9]{4})\", \"mes\" : \"\", \"dia\" : \"\", \"origem\" :  \"Receitas\"', response.text, re.I | re.DOTALL)
anos

['2013', '2014', '2015', '2016', '2017', '2018']

In [15]:
#url = re.findall('var url = springUrl \+ \"(.*?)\"', response.content)[0]

In [16]:
for ano in anos:
    
    # Obtendo a URL de download do ano desejado
    url = '{}{}/{}'.format(TRANSPARENCIA_URL, RECEITA_URL, ano)
    print('Processando a URL: {}'.format(url))
    filename = 'receita_{}.zip'.format(ano)

    # Obtendo informacoes do arquivo a ser baixado
    r = requests.get(url, stream=True)
    total_length = r.headers.get('content-length')
    if total_length is None:
        total_length = 0
    total_length = int(total_length)

    # Download em si do arquivo
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192*4):
            if not chunk:
                log.debug('chunk nulo')
                continue
            f.write(chunk)
            f.flush()
            os.fsync(f.fileno())

    # Descomprimindo o arquivo
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall('Receita/')
    zip_ref.close()

    # Deletando o .zip
    os.remove(filename)

Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2013
Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2014
Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2015
Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2016
Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2017
Processando a URL: http://www.portaltransparencia.gov.br/download-de-dados/receitas/2018


In [17]:
!ls

Apresentacao            Receita                 transparencia.py
Dados                   geckodriver             transparencia_scraper
LICENSE                 geckodriver.log         twitter.ipynb
README.md               transparencia.ipynb     twitter.ipynb - cópia


In [18]:
!ls Receita

2013_Receitas.csv 2015_Receitas.csv 2017_Receitas.csv
2014_Receitas.csv 2016_Receitas.csv 2018_Receitas.csv


# E se usassemos SELENIUM ???

In [14]:
#!pip install -U selenium

In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

### Define o navegador a ser utilizado

In [20]:
driver = webdriver.Firefox()
driver.get(TRANSPARENCIA_URL+RECEITA_URL)

### Espera um tempo para carregar a pagina

In [21]:
time.sleep(15)

### Busca o combo com os anos disponiveis

In [22]:
dados = driver.find_element_by_id("links-anos")
dados

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="dd02918c-b150-7143-872a-867f21e01d85", element="1da9cb50-22ba-4f46-8871-d0e445f0c4f3")>

### Obtem o texto com o HTML retornado

In [23]:
html = dados.get_attribute("innerHTML")
html

'<option value="2018">2018</option><option value="2017">2017</option><option value="2016">2016</option><option value="2015">2015</option><option value="2014">2014</option><option value="2013">2013</option>'

In [20]:
anos = BeautifulSoup(html, "html.parser")

In [21]:
options = anos.find_all("option")
values = [o.get("value") for o in options]
for x in values:
    print(x)

2018
2017
2016
2015
2014
2013


## Basta prosseguir com o Download como feito anteriormente montando a URL de download e baixando arquivo a arquivo